# Inversion of Size Distribution From Data 

<div class="alert alert-warning">

Code is based on Session 4 in the tutorial. This script demonstrates how to use the code in a standalone program.
    
</div>


In [ ]:
using DifferentialMobilityAnalyzers, CSV, Gadfly, DataFrames, Printf

# Load a simple comma delimited text file
df = CSV.read("example_data.csv", DataFrame)

# Setup the DMA
t, p, lpm = 293.15, 940e2, 1.666e-5      # Temperature [K], Pressure [Pa], L min-1 to m3 s-1
r₁, r₂, l = 9.37e-3,1.961e-2,0.44369     # DMA geometry [m]
Λ = DMAconfig(t,p,1lpm,4lpm,r₁,r₂,l,0.0,:+,6,:cylindrical)  
δ = setupDMA(Λ, vtoz(Λ,10000), vtoz(Λ,10), 120)

# Interpolate the data onto the DMA grid
𝕣 = (df,:Dp,:Rcn,δ) |> interpolateDataFrameOntoδ

# Fast inversion with prescribed λ
𝐈, 𝐒, 𝐀, λ =  δ.𝐈, δ.𝐒, δ.𝐀, 0.5
𝕟ⁱⁿᵛ¹ = (𝐀'𝐀 + λ^2𝐈)^(-1) * (𝐀'𝕣  + λ^2 * 𝐒^(-1)*𝕣)

# Slow inversion using λopt from the L-curve method
𝕟ⁱⁿᵛ² = rinv(𝕣.N, δ,λ₁=0.1,λ₂=1.0)

## Plot the inverted data

In [ ]:
df1 = DataFrame(Dp = 𝕟ⁱⁿᵛ¹.Dp, S = 𝕟ⁱⁿᵛ¹.S, Dist = ["𝕟ⁱⁿᵛ¹" for i = 1:length(𝕟ⁱⁿᵛ¹.Dp)])
df2 = DataFrame(Dp = 𝕟ⁱⁿᵛ².Dp, S = 𝕟ⁱⁿᵛ².S, Dist = ["𝕟ⁱⁿᵛ²" for i = 1:length(𝕟ⁱⁿᵛ².Dp)])
df = [df1;df2]
  
xlabels = log10.([10, 100, 500])
colors = ["darkred", "steelblue3", "black"]
p2 = plot(
    df,
    x = :Dp,
    y = :S,
    color = :Dist,
    Geom.step,
    Guide.xlabel("Particle diameter (nm)"),
    Guide.ylabel("dN/dlnD (cm-3)"),
    Guide.xticks(ticks = log10.([10:10:90;100:100:500])),
    Guide.colorkey(; title = ""),
    Scale.x_log10(labels = x -> x in xlabels ? @sprintf("%2i", exp10(x)) : ""),
    Scale.color_discrete_manual(colors...),
    Coord.cartesian(xmin = log10(10), xmax = log10(500), ymin = 0)
)